In [1]:
import numpy

class ObjectiveFunctionCalculator: 
    
    RFRate = 0.000227273
    portfolioPrices = []
    BMPrices = []
    portfolioReturns = []
    BMReturns = []
    BMEReturns = ["notset"]
    RFEReturns = ["notset"]
    negativeBMEReturns = ["notset"]

    def __init__(self, portfolioP, BMP):
        self.portfolioPrices = portfolioP
        self.BMPrices = BMP
        self.portfolioReturns = self.calculateReturns(self.portfolioPrices)
        self.BMReturns = self.calculateReturns(self.BMPrices)
        return
    
    def numpyUsage(self):
        # Example return series one
        returnSeries = [0.0357, -0.0172, 0.1403, -0.0461]
        # Standard deviation
        numpy.std(returnSeries)
        # Variance
        numpy.var(returnSeries)
        # Example return series two
        returnSeries2 = [0.0099, 0.0196, 0.0288, 0.0186]
        # Covariance
        numpy.covar(returnSeries, returnSeries2)
        # Mean convariance
        numpy.mean(numpy.covar(returnSeries, returnSeries2))

    def calculateReturns(self, prices):
        returns = []
        for t in range(len(prices) - 1):
            t = t+1
            returns.append((prices[t] / prices[t-1]) - 1.000)
        return returns

    def validateInternalState(self):
        if len(self.portfolioReturns) == len(self.BMReturns):
            return "true"
        else:
            return "false"

    def getTotalReturn(self, prices):
        return (prices[len(prices)-1] / prices[0]) - 1

    def reset(self):
        self.BMEReturns = ["notset"]
        self.negativeEReturns = ["notset"]
        return

    def setRFRate(self, val):
        self.RFRate = val

    def setBMReturns(self, returns):
        self.BMReturns = returns
        self.reset()
        return

    def getBMReturns(self):
        return self.BMReturns

    def setPortfolioReturns(self, returns):
        self.portfolioReturns = returns
        self.reset()

    def getPortfolioReturns(self):
        return self.portfolioReturns

    # RFEReturns: Risk Free Excess Returns
    def getExcessRFReturns(self):
        if self.RFEReturns[0] == "notset":
            self.RFEReturns = self.calculateExcessRFReturns()
            return self.RFEReturns
        else:
            return self.RFEReturns
    
    # RFEReturns: Risk Free Excess Returns
    def calculateExcessRFReturns(self):
        RFEReturns = []
        for day in range(len(self.portfolioReturns)):
            RFEReturns.append(self.portfolioReturns[day] - 
            self.RFRate)
        return RFEReturns

    # BMEReturns: Benchmark Excess Returns
    def getExcessBMReturns(self):
        if self.validateInternalState() == "true":
            if self.BMEReturns[0] == "notset":
                self.BMEReturns = self.calculateExcessBMReturns()
                return self.BMEReturns
            else:
                return self.BMEReturns
        else:
            print "Correct internal state error and try again"  
            
    # BMEReturns: Benchmark Excess Returns
    def calculateExcessBMReturns(self):
        BMEReturns = []
        for day in range(len(self.portfolioReturns)):
            BMEReturns.append(self.portfolioReturns[day] - self.BMReturns[day])
        return BMEReturns
    
    # BMEReturns: Benchmark Excess Returns
    def getNegativeBMEReturns(self):
        if self.validateInternalState() == "true":
            if self.negativeBMEReturns[0] == "notset":
                self.negativeBMEReturns = self.calculateNegativeBMEReturns()
                return self.negativeBMEReturns
            else:
                return self.negativeBMEReturns
        else:
            print "Correct internal state error and try again"  

    # BMEReturns: Benchmark Excess Returns
    def calculateNegativeBMEReturns(self):
        self.calculateExcessBMReturns()
        negativeEReturns = []
        for ret in self.BMEReturns:
            if ret < 0:
                negativeEReturns.append(ret)
        return negativeEReturns

    #BMReturns: Benchmark returns
    def getPortfolioBeta(self):
        covariances = numpy.cov(self.portfolioReturns, self.BMReturns)
        return numpy.mean(covariances / numpy.var(self.BMReturns))
    
    # RFEReturns: Risk Free Excess Returns
    def getPortfolioSharpeRatio(self):
        meanRFEReturns = numpy.mean(self.RFEReturns)
        stdRFEReturns = numpy.std(self.RFEReturns)
        return meanRFEReturns / stdRFEReturns
    
    # BMEReturns: Benchmark Excess Returns
    def getPortfolioSortinoRatio(self):
        meanBMEReturns = numpy.mean(self.BMEReturns)
        stdNegativeEReturns = numpy.std(self.negativeBMEReturns)
        return meanBMEReturns / stdNegativeEReturns

    # BMEReturns: Benchmark Excess Returns
    def getPortfolioInformationRatio(self):
        meanBMEReturns = numpy.mean(self.BMEReturns)
        stdBMEReturns = numpy.std(self.BMEReturns)
        return meanBMEReturns / stdBMEReturns

    #RFRate: Risk Free Rate
    def getTreynorRatio(self):
        portfolioReturn = self.getTotalReturn(self.portfolioPrices)
        return (portfolioReturn - self.RFRate)/ self.getPortfolioBeta()
    
    # BMEReturns: Benchmark Excess Returns    
    def getPortfolioModiglianiRatio(self):
        infomormatioRatio = self.getPortfolioInformationRatio()
        stdBMEReturns = numpy.std(self.BMEReturns)
        return infomormatioRatio * stdBMEReturns + self.RFRate

In [2]:
portfolioPrices = [56.000,58.000,57.000,65.000,62.000]
benchmarkPrices = [101.000,102.000,104.000,107.000,109.000]
calculator = ObjectiveFunctionCalculator(portfolioPrices,benchmarkPrices) 
print "~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*"
portfolioReturns = calculator.getPortfolioReturns()
print "Portfolio returns = ", portfolioReturns
benchmarkReturns = calculator.getBMReturns()
print "Benchmark returns = ", benchmarkReturns
print "~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*"
riskFreeExcessReturns = calculator.getExcessRFReturns()
print "Excess returns to the risk free rate = ", riskFreeExcessReturns
benchmarkExcessReturns = calculator.getExcessBMReturns()
print "Excess returns to the benchmark = ", benchmarkExcessReturns
negativeExcessReturns = calculator.getNegativeBMEReturns()
print "Negative excess returns to the benchmark = ", negativeExcessReturns
print "~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*"
meanRFEReturns = numpy.mean(calculator.getExcessRFReturns())
print "Mean of excess returns to risk free rate = ", meanRFEReturns
meanBMEReturns = numpy.mean(calculator.getExcessBMReturns())
print "Mean of excess returns to the benchmark = ", meanBMEReturns
stdRFEReturns = numpy.std(calculator.getExcessRFReturns())
print "Stdev of excess returns to risk free rate = ", stdRFEReturns
stdBMEReturns = numpy.std(calculator.getExcessBMReturns())
print "Stdev of excess returns to benchmark = ", stdBMEReturns
print "~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*"
sharpeRatio = calculator.getPortfolioSharpeRatio()
print "Sharpe Ratio using risk free rate = ", sharpeRatio
informationRatio = calculator.getPortfolioInformationRatio()
print "Information Ratio using benchmark = ", informationRatio
sortinoRatio = calculator.getPortfolioSortinoRatio()
print "Sortino Ratio using benchmark = ", sortinoRatio
betaCoefficient = calculator.getPortfolioBeta()
print "Average portfolio beta coefficient = ", betaCoefficient
treynorRatio = calculator.getTreynorRatio()
print "Treynor ratio using risk free rate = ", treynorRatio
M2Ratio = calculator.getPortfolioModiglianiRatio()
print "Modigliani / M2 ratio = ", M2Ratio
print "~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*"

~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Portfolio returns =  [0.03571428571428581, -0.017241379310344862, 0.14035087719298245, -0.0461538461538461]
Benchmark returns =  [0.00990099009900991, 0.019607843137254832, 0.028846153846153744, 0.01869158878504673]
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Excess returns to the risk free rate =  [0.03548701271428581, -0.017468652310344862, 0.14012360419298245, -0.0463811191538461]
Excess returns to the benchmark =  [0.0258132956152759, -0.036849222447599694, 0.1115047233468287, -0.06484543493889283]
Negative excess returns to the benchmark =  [-0.036849222447599694, -0.06484543493889283]
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Mean of excess returns to risk free rate =  0.0279402113608
Mean of excess returns to the benchmark =  0.0089058403939
Stdev of excess returns to risk free rate =  0.0711120469882
Stdev of excess returns to benchmark =  0.0677221699558
~*~